In [1]:
# ===================================================================
# LANGKAH 1: INSTALASI LIBRARY
# Menjalankan ini akan menginstal EasyOCR dan library lain yang diperlukan.
# ===================================================================
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3.10.19:
      Successfully uninstalled nvidia-curand-cu12-10.3.10.19
  Attempt

In [2]:
# ===================================================================
# LANGKAH 2: IMPORT LIBRARY
# ===================================================================
import os
import pandas as pd
import easyocr
from PIL import Image
import torch
import warnings
# Mengabaikan peringatan yang tidak relevan dari PIL
warnings.filterwarnings("ignore", category=UserWarning, module="PIL")

In [3]:
def process_directory_easyocr(directory, label):
    """
    Memproses semua gambar dalam sebuah direktori menggunakan EasyOCR,
    mengekstrak teks, dan mengembalikan DataFrame.
    """
    file_names = []
    ocr_texts = []
    classes = []

    # --- Persiapan: Hitung total file untuk menunjukkan progres ---
    image_files_to_process = []
    print(f"Mencari file gambar di: {directory}")
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                image_files_to_process.append(os.path.join(root, filename))
    
    total_files = len(image_files_to_process)
    if total_files == 0:
        print("-> Tidak ada file gambar yang ditemukan.")
        return pd.DataFrame({'File Name': [], 'Extracted Text': [], 'Class': []})
    
    print(f"-> Total gambar yang akan diproses: {total_files}")

    # --- Inisialisasi EasyOCR Reader ---
    # Inisialisasi ini bisa memakan waktu beberapa saat saat pertama kali dijalankan
    # karena perlu mengunduh model bahasa.
    # Kita akan mendeteksi bahasa Indonesia ('id') dan Inggris ('en').
    use_gpu = torch.cuda.is_available()
    print(f"-> Menggunakan device: {'GPU' if use_gpu else 'CPU'}")
    reader = easyocr.Reader(['id', 'en'], gpu=use_gpu)

    # --- Proses Setiap File Gambar ---
    for i, file_path in enumerate(image_files_to_process):
        filename = os.path.basename(file_path)
        
        try:
            # Tampilkan progres saat ini
            print(f"Memproses file {i + 1}/{total_files}: {filename}")
            
            file_names.append(filename)

            # EasyOCR dapat menerima path file secara langsung
            ocr_result = reader.readtext(file_path, detail=1, paragraph=False)
            
            # Cek apakah EasyOCR menemukan teks
            if ocr_result:
                # Gabungkan semua potongan teks yang ditemukan menjadi satu baris,
                # dipisahkan oleh spasi.
                extracted_text = ' '.join([res[1] for res in ocr_result])
                ocr_texts.append(extracted_text.strip())
            else:
                # Jika tidak ada teks yang terdeteksi, tambahkan string kosong
                ocr_texts.append("")

            # Tambahkan label kelas
            classes.append(label)
            
        except Exception as e:
            print(f"!! ERROR saat memproses {filename}: {e}")
            # Jika terjadi error, catat pesan error agar mudah dilacak
            ocr_texts.append(f"ERROR: {e}")
            classes.append(label)
            continue

    return pd.DataFrame({'File Name': file_names, 'Extracted Text': ocr_texts, 'Class': classes})

In [4]:
# ===================================================================
# LANGKAH 4: EKSEKUSI UTAMA
# ===================================================================

# Tentukan path ke direktori gambar Anda
judi_dir = '/kaggle/input/gamblingdet-id/situsjudiid-full/judi'
normal_dir = '/kaggle/input/gamblingdet-id/situsjudiid-full/non-judi'

# Proses kedua direktori menggunakan fungsi baru
print("\n--- Memproses Direktori Judi ---")
judi_df = process_directory_easyocr(judi_dir, 'judi')

print("\n--- Memproses Direktori Non-Judi ---")
normal_df = process_directory_easyocr(normal_dir, 'non-judi')

# Gabungkan kedua DataFrame menjadi satu
final_df = pd.concat([judi_df, normal_df], ignore_index=True)

# Tampilkan beberapa baris pertama dan terakhir dari hasil gabungan
print("\n--- Hasil Gabungan (Contoh) ---")
if not final_df.empty:
    print(final_df.head().to_string())
    print("...")
    print(final_df.tail().to_string())
else:
    print("DataFrame akhir kosong.")

print(f"\nTotal baris data yang dihasilkan: {len(final_df)}")

# Simpan DataFrame akhir ke file CSV
output_path = '/kaggle/working/text_ocr_easyocr.csv'
final_df.to_csv(output_path, index=False, encoding='utf-8')

print(f"\nProses selesai. Hasil telah disimpan ke: {output_path}")



--- Memproses Direktori Judi ---
Mencari file gambar di: /kaggle/input/gamblingdet-id/situsjudiid-full/judi
-> Total gambar yang akan diproses: 1950
-> Menggunakan device: GPU
Memproses file 1/1950: ahoyamigo.com_home.png
Memproses file 2/1950: asgard789.bet.png
Memproses file 3/1950: bola442.monster.png
Memproses file 4/1950: maniac-ihokibet.com_page1.png
Memproses file 5/1950: ketohour.com_page4.png
Memproses file 6/1950: agenclassicgames.webflow.io.png
Memproses file 7/1950: bolakoh.com.png
Memproses file 8/1950: manjabet1.life_page4.png
Memproses file 9/1950: aksessingawin.us_home.png
Memproses file 10/1950: air-pegunungan.pro_page5.png
Memproses file 11/1950: aguaonline.net_home.png
Memproses file 12/1950: nikeshopjapan.com.png
Memproses file 13/1950: boladoki.com.png
Memproses file 14/1950: angkosimba4d.com_home.png
Memproses file 15/1950: bolaidp88.site.png
Memproses file 16/1950: judetabet89.lol_home.png
Memproses file 17/1950: lulu88casino.com_page5.png
Memproses file 18/1950

In [5]:
final_df.head()

,File Name,Extracted Text,Class
0,ahoyamigo.com_home.png,KASKUSTOTO LOGIN KASKUSTOTO oiocom KASKUSTOTO ...,judi
1,asgard789.bet.png,SUCKBET kunusn Tusluju Unajv ]acipısı aüns I...,judi
2,bola442.monster.png,AsF User Name Password Code 6636 LOGIN Lela442...,judi
3,maniac-ihokibet.com_page1.png,Download APK Live Chat Hubungı Kamı Lupa Kata ...,judi
4,ketohour.com_page4.png,ketohour Beranda 7 @ Q Subscribe Beranda Situs...,judi
